<a href="https://colab.research.google.com/github/luciozarazaga/SistemasDeRecomendacion/blob/main/Practico_Sist_Rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.sparse import csr_matrix

## Grupo 4

## Integrantes


*   Fernando Vázquez
*   Gaspar García Bazán
*   Lucio A. Zarazaga
*   Pablo Estrada



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
artist_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/artists.dat', sep='\t')
artist_data= artist_data.rename(columns={'id': 'artistID'})
artist_data.head()

,artistID,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [ ]:
user_artist_plays = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lastfm/user_artists.dat", sep="\t")
user_artist_plays.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
tags=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lastfm/tags.dat', sep='\t', engine='python')
tags.head()

,tagID,tagValue
0,1,metal
1,2,alternative metal
2,3,goth rock
3,4,black metal
4,5,death metal


In [ ]:
user_friends=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lastfm/user_friends.dat', sep='\t')
user_friends.head()

,userID,friendID
0,2,275
1,2,428
2,2,515
3,2,761
4,2,831


In [ ]:
user_taggedartists=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lastfm/user_taggedartists.dat', sep='\t')
user_taggedartists.head()

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009


## 1 - Análisis Exploratorio de datos

¿Cuales fueron los artistas más escuchados?

In [ ]:
top_artist= pd.merge(left=user_artist_plays,right=artist_data, left_on='artistID', right_on='artistID')
top_artist.groupby(by='name').sum().sort_values(by='weight', ascending=False).head(10)

,userID,artistID,weight
name,,,
Britney Spears,536585,150858,2393140
Depeche Mode,282739,20304,1301308
Lady Gaga,621257,54379,1291387
Christina Aguilera,397259,118844,1058405
Paramore,416630,198702,963449
Madonna,424473,28743,921198
Rihanna,493104,139392,905423
Shakira,312770,223619,688529
The Beatles,516563,108960,662116


La artista más escuchada fue Britney Spears con 2393140 reproducciones. Le siguen Depeche Mode y Lady Gaga con 1301308 y 1291387 reproducciones respectivamente.

¿Como es distribución de cantidad de listens per user?

In [ ]:
user_artist_plays.groupby('userID').sum().sort_values('weight', ascending=False).head(10)

,artistID,weight
userID,,
757,104544,480039
2000,354767,468409
1418,224538,416349
1642,551588,388251
1094,201719,379125
1942,373698,348527
2071,133895,338400
2031,106389,329980
514,93246,329782


El usuario de ID 757 es el que más reproducciones realizo

¿Es posible ver el género más escuchado?

In [ ]:
# Se decidió trabajar con el archivo "user_taggedartists.dat"
df_tags2 = tags.copy()
tagID_tagValue = df_tags2.set_index('tagID')['tagValue'].to_dict()

serie_top_ranking_tag = user_taggedartists['tagID'].value_counts(ascending=False)


df_top_ranking_tag = serie_top_ranking_tag.to_frame()
df_top_ranking_tag = df_top_ranking_tag.reset_index()

df_top_ranking_tag.columns = ['tagID','total_listened']
df_top_ranking_tag['tagID'] = df_top_ranking_tag['tagID'].replace(tagID_tagValue)
df_top_ranking_tag.columns = ['type_of_music','total_listened']
df_top_ranking_tag.head(10)

,type_of_music,total_listened
0,rock,7503
1,pop,5418
2,alternative,5251
3,electronic,4672
4,indie,4458
5,female vocalists,4228
6,80s,2791
7,dance,2739
8,alternative rock,2631
9,classic rock,2287


## 2 Sistema de Recomendación

##2A Filtrado colaborativo

In [ ]:
# convertimos la columna weight a float
user_artist_plays['weight']=user_artist_plays['weight'].astype(float)

In [ ]:
user_artist_plays.dtypes

userID        int64
artistID      int64
weight      float64
dtype: object

In [ ]:
#normalización de los datos
user_artist_plays['weight_scaled']=(user_artist_plays['weight']-user_artist_plays['weight'].mean()) /user_artist_plays['weight'].std()
user_plays=  user_artist_plays.drop('weight', 1)
user_plays.head()

,userID,artistID,weight_scaled
0,2,51,3.502167
1,2,52,2.917573
2,2,53,2.827205
3,2,54,2.547037
4,2,55,2.195961


In [ ]:
len(user_plays)

92834

In [ ]:
user_plays.weight_scaled.describe()

count    9.283400e+04
mean    -1.496697e-17
std      1.000000e+00
min     -1.983951e-01
25%     -1.701384e-01
50%     -1.293528e-01
75%     -3.498605e-02
max      9.382099e+01
Name: weight_scaled, dtype: float64

In [ ]:
#eliminamos los usuarios con num_raitings menor a 5
min_ratings=5
grouped_df=user_plays.groupby('artistID', as_index=False).count().iloc[:,lambda df: [0,2]].sort_values('weight_scaled', ascending=False)
grouped_df= grouped_df.loc[grouped_df['weight_scaled'] >= min_ratings]
grouped_df.columns=['artistID', 'num_raitings']
filtered_df=user_plays.merge(grouped_df, how='inner', on = 'artistID') #datos filtrados

In [ ]:
filtered_df

,userID,artistID,weight_scaled,num_raitings
0,2,51,3.502167,111
1,4,51,-0.137883,111
2,27,51,-0.176003,111
3,28,51,-0.195996,111
4,62,51,-0.057911,111
...,...,...,...,...
71421,1479,14692,-0.108294,6
71422,1737,14692,-0.124821,6
71423,1759,14692,0.072443,6
71424,1890,14692,-0.159475,6


In [ ]:
filtered_df.describe()

,userID,artistID,weight_scaled,num_raitings
count,71426.000000,71426.000000,71426.000000,71426.000000
mean,1032.297861,1527.646627,0.026171,111.226640
std,609.305190,1878.657582,1.130231,130.262313
min,2.000000,2.000000,-0.198395,5.000000
25%,498.000000,321.000000,-0.166673,19.000000
50%,1024.000000,808.000000,-0.122155,60.000000
75%,1559.000000,1936.750000,-0.017392,150.000000
max,2100.000000,14692.000000,93.820991,611.000000


In [ ]:
filtered_df=filtered_df.fillna(value=0)

In [ ]:
combined_data= pd.merge(left=filtered_df,right=artist_data, left_on='artistID', right_on='artistID')
combined_data=combined_data.drop(['url','pictureURL'],axis= 1)
combined_data.sample(5)

,userID,artistID,weight_scaled,num_raitings,name
57397,96,2552,-0.189065,10,Culture Club
36759,635,868,-0.182667,139,The Doors
41583,1360,1048,-0.161875,130,The Kooks
21842,1656,425,-0.108827,128,Yeah Yeah Yeahs
60123,1329,3003,-0.144547,6,Duke Ellington


In [ ]:
combined_data.weight_scaled.isnull().values.any()

False

In [ ]:
#aqui se arma la matriz usuario contenido
rating_crosstab = combined_data.pivot_table(values='weight_scaled', index='userID', columns='name', fill_value=0)
rating_crosstab.head()

name,#####,(hed) Planet Earth,*NSYNC,+44,...And You Will Know Us by the Trail of Dead,10cc,12 Stones,2H Company,2NE1,2PM,2Pac,2ois,3 Doors Down,30 Seconds to Mars,311,36 Crazyfists,3OH!3,4minute,5'nizza,50 Cent,65daysofstatic,7раса,808 State,8mm,:wumpscut:,A Day to Remember,A Fine Frenzy,A Flock of Seagulls,A Perfect Circle,A Rocket to the Moon,A Silver Mt. Zion,A Skylit Drive,A Static Lullaby,A Tribe Called Quest,A Wilhelm Scream,A*Teens,ABBA,ABC,AC/DC,AFI,...,Ляпис Трубецкой,Мои Ракеты Вверх,Мумий Тролль,Ногу Свело!,Ночные Снайперы,Океан Ельзи,П.Т.В.П.,Пикник,Пилот,Психея,Сергей Лазарев,Слот,Сплин,いきものがかり,アンティック-珈琲店-,ギルガメッシュ,ルルティア,下村陽子,中島美嘉,久石譲,倖田來未,坂本龍一,大塚愛,宇多田ヒカル,安室奈美恵,嵐,平沢進,梶浦由記,植松伸夫,椎名林檎,浜崎あゆみ,浜渦正志,菅野よう子,譚盾,近藤浩治,雅-MIYAVI-,鬼束ちひろ,동방신기,소녀시대,티아라
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151881,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.19493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#same
X = rating_crosstab.T

Utilizamos las herramientas provistas por surprise para crear el sistema de recomendación.

In [ ]:
pip install surprise

     |████████████████████████████████| 11.8MB 6.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618279 sha256=08593efd4a61fff1061276c24290c2c44e6a8aaac16320d362a95ec84b0bd4c6
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate, train_test_split
from surprise import SVD, accuracy
from surprise import SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [ ]:
reader=Reader(rating_scale=(combined_data.weight_scaled.min(), combined_data.weight_scaled.max()))
df_user=Dataset.load_from_df(combined_data[['userID', 'artistID', 'weight_scaled']], reader) #train data

In [ ]:
benchmark = []
# Iterate over collaborative filtering algorithms
for algorithm in [ SlopeOne(), NMF(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, df_user, measures=['RMSE', 'MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SlopeOne,1.113203,0.231090,0.349486,0.763298
KNNWithMeans,1.116841,0.234086,0.317002,2.468436
KNNBaseline,1.117887,0.218473,0.391579,2.872973
KNNBasic,1.129748,0.227569,0.285230,2.275802
KNNWithZScore,1.235631,0.222554,0.389387,2.652049
CoClustering,1.514296,0.277525,1.286755,0.164276
NMF,1.870123,0.315135,3.313770,0.169029


El menor RMSE esta dado por KNNBaseline, por lo tanto es el que se usará para recomendar las peliculas.

In [ ]:
## Using collaborative filtering baseline
algo=KNNBaseline()
perf=cross_validate(algo, df_user, measures=['RMSE','MAE'], cv=3, verbose=False)

print(perf)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
{'test_rmse': array([1.13892491, 1.26550653, 0.95510341]), 'test_mae': array([0.21619006, 0.21878739, 0.21914198]), 'fit_time': (0.37593698501586914, 0.3793175220489502, 0.4011204242706299), 'test_time': (2.7355940341949463, 2.8064522743225098, 2.7097318172454834)}


Como primer paso entrenamos el modelo

In [ ]:
#Entrenamos el modelo de recomendación: Paso 1
def getSimModle():
  trainset=df_user.build_full_trainset()
  #usamos pearson_baseline para calcular la similitud entre los artistas 
  sim_options={'name':'pearson_baseline', 'user_based':False}
  #usamos KNNBaseline
  algo=KNNBaseline(sim_options=sim_options)
  #entrenamos el modelo
  algo.fit(trainset)
  return algo


In [ ]:
#creacion de diccionarios: Paso2
name_to_rid = dict(zip(combined_data['name'],combined_data['artistID']))
rid_to_name = dict(zip(combined_data['artistID'],combined_data['name']))

### creamos el sistema de recomendacion

In [ ]:

# Recomendación de artistas relacionados basados en el modelo definido anteriormente: Paso 3
def showSimilarArtist(algo, rid_to_name, target_artist):
    # Obtenemos la raw_id del artista seleccionado
    target_artist_raw_id = name_to_rid.get(target_artist)
    
    #convertimos la raw_id del artista a la ID interna del modelo
    target_artist_inner_id = algo.trainset.to_inner_iid(target_artist_raw_id)
    
    #Obtenemos los artistas recomendados por el modelo, aquí solo 10.
    target_artist_neighbors = algo.get_neighbors(target_artist_inner_id, 10)
    
    #La id interna del modelo es convertida a la id actual del artista
    neighbors_raw_ids = [algo.trainset.to_raw_iid(inner_id) for inner_id in target_artist_neighbors]
    #Obtenemos una lista de artist-id o una lista de recomendaciones
    neighbors_artist = [rid_to_name[raw_id] for raw_id in neighbors_raw_ids]
    print('The 10 nearest neighbors of the artist are:')
    for artist in neighbors_artist:
       print(artist)
    

In [ ]:
#entrenamos el modelo de recomendación
algo=getSimModle()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
#mostramos los artistas relacionados
target_artist = 'AC/DC'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
Scorpions
Aerosmith
Queen
Black Label Society
Motörhead
KISS
The Strokes
Jimi Hendrix
The Killers
R.E.M.


In [ ]:
#mostramos los artistas relacionados
target_artist = 'Ramones'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
Black Sabbath
Matanza
Blur
Judas Priest
Rise Against
Nirvana
Pearl Jam
The Clash
Arcade Fire
Echo & The Bunnymen


In [ ]:
#mostramos los artistas relacionados
target_artist = 'Justin Bieber'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
Black Eyed Peas
Mariah Carey
Selena Gomez & the Scene
Paramore
Ke$ha
McFly
Kelly Clarkson
Hilary Duff
3OH!3
OneRepublic


In [ ]:
#mostramos los artistas relacionados
target_artist = 'The Beatles'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
Amy Winehouse
The Rolling Stones
Snow Patrol
Travis
Bloc Party
Oasis
Genesis
John Lennon
The White Stripes
The Fray


In [ ]:
#mostramos los artistas relacionados
target_artist = 'Joy Division'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
The Smiths
Arcade Fire
Sonic Youth
Coldplay
New Order
The xx
Martin L. Gore
Metallica
White Lies
Kaiser Chiefs


In [ ]:
##mostramos los artistas relacionados
target_artist = 'Muse'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
Avril Lavigne
Good Charlotte
Ke$ha
Arctic Monkeys
Deftones
Lady Gaga
The Veronicas
The Prodigy
Rise Against
Justin Timberlake


In [ ]:
#mostramos los artistas relacionados
target_artist = 'Radiohead'
showSimilarArtist(algo ,rid_to_name, target_artist)

The 10 nearest neighbors of the artist are:
Pixies
Oasis
Red Hot Chili Peppers
Death Cab for Cutie
The Cranberries
The Who
Interpol
Keane
The Mars Volta
David Bowie


## Parte 2b

## 2b.1

In [ ]:
#buscamos la ID de esos artistas que dejaremos afuera (10)
array=['Taylor Swift', 'Madonna', 'Evanescense', 'Radiohead', 'Muse', 'Depeche Mode', 'Paramore', 'The Beatles' , 'Shakira', 'Britney Spears']
artist_data.loc[artist_data['name'].isin(array)]


,artistID,name,url,pictureURL
61,67,Madonna,http://www.last.fm/music/Madonna,http://userserve-ak.last.fm/serve/252/340387.jpg
66,72,Depeche Mode,http://www.last.fm/music/Depeche+Mode,http://userserve-ak.last.fm/serve/252/75022.jpg
148,154,Radiohead,http://www.last.fm/music/Radiohead,http://userserve-ak.last.fm/serve/252/8461967.jpg
184,190,Muse,http://www.last.fm/music/Muse,http://userserve-ak.last.fm/serve/252/416514.jpg
221,227,The Beatles,http://www.last.fm/music/The+Beatles,http://userserve-ak.last.fm/serve/252/2588646.jpg
283,289,Britney Spears,http://www.last.fm/music/Britney+Spears,http://userserve-ak.last.fm/serve/252/60126439...
338,344,Taylor Swift,http://www.last.fm/music/Taylor+Swift,http://userserve-ak.last.fm/serve/252/45447333...
492,498,Paramore,http://www.last.fm/music/Paramore,http://userserve-ak.last.fm/serve/252/35837991...
695,701,Shakira,http://www.last.fm/music/Shakira,http://userserve-ak.last.fm/serve/252/52116105...


In [ ]:
#con la ID quitamos la informacion de los artistas de user_taggedartist
array_id=[67, 72, 154, 190, 227, 289,  344, 498, 701]
drop_artist_data = user_taggedartists.loc[user_taggedartists['artistID'].isin(array_id)] #datos de los artistas que dejamos fuera
user_taggedartists = user_taggedartists.loc[~user_taggedartists['artistID'].isin(array_id)] #artistas que quedan

In [ ]:
#con la ID quitamos la información de los artistas de artist_data
drop_artist = artist_data.loc[artist_data['artistID'].isin(array_id)] #datos de los artistas que dejamos fuera
useful_artist= artist_data.loc[~artist_data['artistID'].isin(array_id)] #artistas que quedan
useful_artist.sample(5)

,artistID,name,url,pictureURL
1936,1951,Pomegranates,http://www.last.fm/music/Pomegranates,http://userserve-ak.last.fm/serve/252/53187247...
15389,16189,The Priests,http://www.last.fm/music/The+Priests,http://userserve-ak.last.fm/serve/252/31599295...
8035,8228,Дети Picasso,http://www.last.fm/music/%D0%94%D0%B5%D1%82%D0...,http://userserve-ak.last.fm/serve/252/165404.jpg
7574,7736,The Novellos,http://www.last.fm/music/The+Novellos,http://userserve-ak.last.fm/serve/252/36057159...
5327,5449,Nosso Trio,http://www.last.fm/music/Nosso+Trio,http://userserve-ak.last.fm/serve/252/7745777.jpg


In [ ]:
user_taggedartists=user_taggedartists.drop(['userID','day', 'month', 'year'],axis= 1)

In [ ]:
useful_artist=useful_artist.drop(['url', 'pictureURL'],axis=1)

##2b.2

In [ ]:

content_vector=pd.merge(left=user_taggedartists,right=tags, left_on='tagID', right_on='tagID')
content_vector.sample(5)

,artistID,tagID,tagValue
180034,728,12209,other great tracks
48826,3027,1,metal
20396,1001,25,80s
146857,13688,2115,acoustic rock
135117,3473,1030,1986


In [ ]:
content_vector['tagValue'].value_counts().head(10)

rock                7229
pop                 5116
alternative         5051
electronic          4511
indie               4359
female vocalists    4030
80s                 2701
dance               2588
alternative rock    2491
classic rock        2199
Name: tagValue, dtype: int64

In [ ]:
len(content_vector)

181245

In [ ]:
#eliminamos los tags con apariciones menor a 2100 para reducir los generos a 10
sub_df = content_vector[content_vector.groupby('tagValue').tagValue.transform('count')>2100].copy() 

In [ ]:
sub_df['tagValue'].value_counts()

rock                7229
pop                 5116
alternative         5051
electronic          4511
indie               4359
female vocalists    4030
80s                 2701
dance               2588
alternative rock    2491
classic rock        2199
Name: tagValue, dtype: int64

In [ ]:
len(sub_df)

40275

In [ ]:
df_final= pd.merge(left=sub_df,right=useful_artist, left_on='artistID', right_on='artistID')
df_final.head()

,artistID,tagID,tagValue,name
0,52,18,electronic,Morcheeba
1,52,18,electronic,Morcheeba
2,52,18,electronic,Morcheeba
3,52,18,electronic,Morcheeba
4,52,18,electronic,Morcheeba


In [ ]:
df_final=df_final.groupby(['artistID', 'name'], as_index=False)['tagValue'].apply(','.join)
df_final.sample(5)

,artistID,name,tagValue
4549,11234,Jarvis Cocker,"pop,alternative,alternative,alternative"
4883,12750,Muscles,"electronic,dance"
3193,6556,The Lemonheads,"alternative rock,indie"
6033,18363,Cassandra Wilson,"female vocalists,female vocalists,female vocal..."
5381,15036,Bruce Hornsby,"80s,80s"


##2.3

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
# Initialize tfidf vectorizer
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,1),stop_words='english', min_df=0)
# Fit and transform 
tfidf_matrix = tf.fit_transform(df_final['tagValue'])

In [ ]:
tfidf_matrix.shape

(6102, 10)

In [ ]:
cos_similarities = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [ ]:
df_final_2=df_final.reset_index()
art_name=df_final_2['name']
indices=pd.Series(df_final_2.index, index=df_final_2['name'])

In [ ]:
def get_recommendations(artist_name): #armamos un sistema de recomendacion basado en contenido
  idx=indices[artist_name]
  sim_scores=list(enumerate(cos_similarities[idx]))
  sim_scores=sorted(sim_scores, key=lambda x: x[1], reverse= True)
  sim_scores=sim_scores[1:31]
  artist_indices= [i[0] for i in sim_scores]
  return art_name.iloc[artist_indices]


In [ ]:
get_recommendations('Nirvana').head(10)

1412                        Creed
287         The Smashing Pumpkins
1191                   Audioslave
1589                     Anberlin
1956               Counting Crows
273     Móveis Coloniais de Acaju
330            30 Seconds to Mars
124           My Chemical Romance
3682                         Live
107             Poets of the Fall
Name: name, dtype: object

In [ ]:
get_recommendations('The Strokes').head(10)

2260         Cold War Kids
139          Kings of Leon
2530                   Ash
91           Stereophonics
140            The Killers
4786         Cheap Mondays
5203      Sleeping at Last
5259           Ecos Falsos
1954        Phantom Planet
2615    Ocean Colour Scene
Name: name, dtype: object

In [ ]:
def hybrid(artist_id, tag):
  idx=indices[arist_name]
  ar_id=links_df
